In [1]:
import pandas as pd
import numpy as np
import pickle
import gensim

In [2]:
with open("data/train.pkl", 'rb') as f:
    train = pickle.load(f) # fix this to find out what was returned and what wasn't
    
trip_nums, X_dicts = zip(*train)
trip_nums = pd.Series(trip_nums)
df = pd.read_csv("data/train.csv")
trip_types = df[['TripType', 'VisitNumber']].groupby('VisitNumber').agg("mean")
print(len(trip_nums), len(trip_types))

95674 95674


 - *try taking first four digits of the UPC*, could be shitty
 - Figure out a way to integrate returns into everything (999)
 - Fix the way the returns are stored in our vocabulary

In [4]:
with open("data/train.pkl", 'rb') as f:
    train = pickle.load(f)
with open("data/test.pkl", 'rb') as f:
    test = pickle.load(f)

train_trip_nums, train_X_dicts = zip(*train)
df = pd.read_csv("data/train.csv")
y = df[['TripType', 'VisitNumber']].groupby('VisitNumber').agg("mean")
test_trip_nums, test_X_dicts = zip(*test)

new_X = []
for row in list(X_dicts):
    add_values = add_feature(row.items())
    row = {**row, **add_values}
    bad_keys = list(filter(positive_feature, row.keys()))
    for key in bad_keys:
        del row[key]
    new_X.append(row)

NameError: name 'add_feature' is not defined

In [56]:
def positive_feature(key):
    if not key.endswith("D") and not key.endswith("F") and not key.startswith("U"):
        return True
    
    else:
        return False
    
def add_feature(sentence):
    feats = {}
    for k, v in sentence:
        if k.endswith("U"):
            if len(k) > 12:
                feats['U_lens_gt_16'] = 1
            else:
                feats['U_lens_lt_16'] = 1
    return feats

new_X = []
for row in list(X_dicts):
    add_values = add_feature(row.items())
    row = {**row, **add_values}
    bad_keys = list(filter(positive_feature, row.keys()))
    for key in bad_keys:
        del row[key]
    new_X.append(row)
new_X[:5]

[{'-1_1000.0_F': 1,
  '-1_FINANCIAL SERVICES_D': 1,
  '1000.0_F': 1,
  'FINANCIAL SERVICES_D': 1,
  'U_lens_gt_16': 1},
 {'1_4504.0_F': 1,
  '1_8931.0_F': 1,
  '1_PERSONAL CARE_D': 1,
  '1_SHOES_D': 1,
  '4504.0_F': 1,
  '8931.0_F': 1,
  'PERSONAL CARE_D': 1,
  'SHOES_D': 1,
  'U_lens_gt_16': 1},
 {'-1_3504.0_F': 1,
  '-1_3565.0_F': 1,
  '-1_PAINT AND ACCESSORIES_D': 2,
  '1001.0_F': 1,
  '1017.0_F': 3,
  '1018.0_F': 1,
  '1028.0_F': 1,
  '1305.0_F': 1,
  '1_1017.0_F': 1,
  '1_1028.0_F': 1,
  '1_2801.0_F': 1,
  '1_2802.0_F': 1,
  '1_3300.0_F': 1,
  '1_4415.0_F': 2,
  '1_4501.0_F': 1,
  '1_4606.0_F': 1,
  '1_6009.0_F': 1,
  '1_7.0_F': 1,
  '1_707.0_F': 1,
  '1_DSD GROCERY_D': 1,
  '1_HOUSEHOLD CHEMICALS/SUPP_D': 1,
  '1_MEAT - FRESH & FROZEN_D': 1,
  '1_PAINT AND ACCESSORIES_D': 8,
  '1_PETS AND SUPPLIES_D': 1,
  '1_nan_D': 1,
  '1_nan_F': 1,
  '2801.0_F': 1,
  '2802.0_F': 1,
  '2_1017.0_F': 2,
  '2_1018.0_F': 1,
  '2_1305.0_F': 1,
  '2_3504.0_F': 1,
  '2_3565.0_F': 2,
  '2_DAIRY_D': 1,

In [116]:
NX = []
import re

for row in new_X:

    sentence = []
    negative_words = []
    for word, count in row.items():
        w1 = word.replace(",", "").replace(" ", "_")
        negative_word = re.search(r"-(\d+)_(.*)", w1)
        if negative_word:
            cnt = negative_word.group(1)
            g = negative_word.group(2)
            if len(g) > 1:
                for x in range(int(cnt)):
                    negative_words.append(g)
        w = [w1]
        sentence += w*count
    for w in negative_words:
        try:
            sentence.remove(w)
        except ValueError:
            pass # sometimes they aren't correct

    NX.append(sentence)
NX[:5]

[['-1_FINANCIAL_SERVICES_D', 'U_lens_gt_16', '-1_1000.0_F'],
 ['1_PERSONAL_CARE_D',
  '1_4504.0_F',
  '1_SHOES_D',
  '8931.0_F',
  'U_lens_gt_16',
  '4504.0_F',
  '1_8931.0_F',
  'PERSONAL_CARE_D',
  'SHOES_D'],
 ['nan_F',
  '1_4606.0_F',
  '-1_3565.0_F',
  '1_7.0_F',
  '1_2802.0_F',
  '7.0_F',
  '1_1028.0_F',
  'DSD_GROCERY_D',
  'U_lens_lt_16',
  'PETS_AND_SUPPLIES_D',
  'PETS_AND_SUPPLIES_D',
  '6009.0_F',
  '2_DAIRY_D',
  '2801.0_F',
  '1_6009.0_F',
  '707.0_F',
  '2_1305.0_F',
  '1017.0_F',
  '1017.0_F',
  '1017.0_F',
  '3_PETS_AND_SUPPLIES_D',
  '1028.0_F',
  '4501.0_F',
  '2_1018.0_F',
  '4606.0_F',
  '1018.0_F',
  '1_1017.0_F',
  '1_4415.0_F',
  '1_4415.0_F',
  '1_2801.0_F',
  '4415.0_F',
  '4415.0_F',
  '-1_3504.0_F',
  '2_1017.0_F',
  '2_1017.0_F',
  '1305.0_F',
  'U_lens_gt_16',
  '2802.0_F',
  '1_3300.0_F',
  '3300.0_F',
  '1_MEAT_-_FRESH_&_FROZEN_D',
  '2_3565.0_F',
  '2_3565.0_F',
  'DAIRY_D',
  '1_DSD_GROCERY_D',
  '1001.0_F',
  'nan_D',
  'HOUSEHOLD_CHEMICALS/SUPP_D',
 

In [117]:
model = gensim.models.Word2Vec(NX, size=100, window=10, min_count=5, sg=0, workers=4)

In [130]:
model.most_similar(negative=['1_115.0_F',
  'U_lens_gt_16',
  'CANDY_TOBACCO_COOKIES_D',
  '1_DSD_GROCERY_D',
  '1_DSD_GROCERY_D',
  '1_2008.0_F',
  '1_7950.0_F',
  'DSD_GROCERY_D',
  'DSD_GROCERY_D',
  '2008.0_F',
  '7950.0_F',
  '115.0_F',
  '1_CANDY_TOBACCO_COOKIES_D'], topn=10)

[('5462.0_F', 0.37320587038993835),
 ('1_6410.0_F', 0.36458951234817505),
 ('-1_3030.0_F', 0.3536050319671631),
 ('4208.0_F', 0.3405967652797699),
 ('5830.0_F', 0.33876028656959534),
 ('1_2151.0_F', 0.3342741131782532),
 ('1_3682.0_F', 0.3333727717399597),
 ('6313.0_F', 0.33083099126815796),
 ('7654.0_F', 0.3288276791572571),
 ('1_1636.0_F', 0.317170113325119)]

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
dv = DictVectorizer()
X = dv.fit_transform(list(new_X))
X

In [144]:
vc = model.vocab.keys()
features = []
for cnt, sent in enumerate(NX):
    s = [word for word in sent if word in vc]
    pos = model.most_similar(positive=s)
    neg = model.most_similar(negative=s)
    if cnt % 10000 == 0: print(str(cnt), "done")
    features.append((s, pos, neg))
print(features[:10])

0 done
10000 done
20000 done
30000 done
40000 done
50000 done
60000 done
70000 done
80000 done
90000 done
[([('-1_279.0_F', 0.7166111469268799), ('-1_3300.0_F', 0.7010934352874756), ('-1_OPTICAL_-_FRAMES_D', 0.6882380247116089), ('-1_2210.0_F', 0.6643586158752441), ('-1_4010.0_F', 0.6555976271629333), ('1_2022.0_F', 0.6510748863220215), ('-1_4503.0_F', 0.6502223014831543), ('-1_BEAUTY_D', 0.6434401273727417), ('-1_4650.0_F', 0.6309057474136353), ('6215.0_F', 0.6212536096572876)], [('3790.0_F', 0.46924012899398804), ('2_3181.0_F', 0.46200835704803467), ('3_3953.0_F', 0.4445602595806122), ('1_8427.0_F', 0.444318950176239), ('2_4150.0_F', 0.4391149878501892), ('1_7950.0_F', 0.43866604566574097), ('2_4602.0_F', 0.43725109100341797), ('9102.0_F', 0.43361178040504456), ('2_6160.0_F', 0.4265710711479187), ('2_1027.0_F', 0.4201729893684387)]), ([('-1_SHOES_D', 0.732521653175354), ('1348.0_F', 0.7008152008056641), ('1376.0_F', 0.6646987199783325), ('2630.0_F', 0.6636745929718018), ('1_1446.0_F'